In [1]:
"""
Feature : cancer
Extract from diagnoses_icd using ICD codes
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load diagnoses
diagnoses = pd.read_csv(
    os.path.join(hosp_path, "diagnoses_icd.csv"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"]
)
diagnoses["icd_code"] = diagnoses["icd_code"].astype(str)
diagnoses["icd_version"] = diagnoses["icd_version"].astype(int)

# ICD-9 Cancer Ranges
icd9_cancer_ranges = [
    (140, 172),
    (174, 195),
    (196, 199),
    (200, 208),
    (209, 209)
]

def is_icd9_cancer(code):
    if not code.isdigit():
        return False
    try:
        num = int(code)
        return any(low <= num <= high for low, high in icd9_cancer_ranges)
    except:
        return False

# ICD-10 Cancer Prefix
def is_icd10_cancer(code):
    return code.startswith("C")

# Apply cancer detection (exactly like notebook - no break statements)
def detect_cancer(group):
    cancer = False
    for code, ver in zip(group["icd_code"], group["icd_version"]):
        if ver == 9 and is_icd9_cancer(code):
            cancer = True
        if ver == 10 and is_icd10_cancer(code):
            cancer = True
    return pd.Series({"cancer": cancer})

cancer_df = (
    diagnoses
    .groupby(["subject_id", "hadm_id"])
    .apply(detect_cancer, include_groups=False)
    .reset_index()
)

# Merge with dataset
df = df.merge(
    cancer_df,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values with False
df["cancer"] = df["cancer"].fillna(False).astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nFeature 'cancer' added.")
print(f"Value counts:\n{df['cancer'].value_counts()}")
print(f"Total rows: {len(df)}")
print(f"True count: {df['cancer'].sum()}")
print(f"False count: {(~df['cancer']).sum()}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")

C:\Users\Coditas\AppData\Local\Temp\ipykernel_5648\4162537091.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["cancer"] = df["cancer"].fillna(False).astype(bool)



Feature 'cancer' added.
Value counts:
cancer
False    144310
True      13710
Name: count, dtype: int64
Total rows: 158020
True count: 13710
False count: 144310
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 42)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "cancer"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)